Install and import the relevant libraries:

In [1]:
import sys
import pandas as pd
import numpy as np

!{sys.executable} -m pip install requests
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install geopy
!{sys.executable} -m pip install folium

import requests
from geopy.geocoders import Nominatim
import folium
import matplotlib.colors as colors
import matplotlib.cm as cm

### Part 1: Housing data

Upload the .csv file containing data on median home prices:

In [2]:
housing = pd.read_csv('Median Home Prices by Borough, Mexico City.csv')
housing

,Borough,Median price
0,Tlalpan,4834600
1,Tláhuac,2092080
2,Venustiano Carranza,2662340
3,Cuauhtémoc,4208070
4,Iztapalapa,2226720
5,Azcapotzalco,2881370
6,Benito Juárez,5188090
7,Cuajimalpa de Morelos,11939500
8,Iztacalco,2735320
9,Gustavo A. Madero,2660400


Import the .txt file that has the boundaries of each borough in Mexico City:

In [3]:
f = open('GeoJSON Boroughs.txt', 'r')
boroughs_geo = f.read()

Use the .txt file and the housing data to make a Chloropleth map of Mexico City:

In [7]:
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode('Mexico City')
latitude = location.latitude
longitude = location.longitude
boroughs_map = folium.Map(location=[latitude - .1, longitude], zoom_start=9, tiles='Mapbox Bright')

boroughs_map.choropleth(
    geo_data=boroughs_geo,
    data = housing,
    columns=['Borough', 'Median price'],
    key_on='feature.properties.nomgeo',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
)

boroughs_map

### Part 2: Metro data

Upload data on the names and GPS coordinates for each metro stop in the city:

In [8]:
metro = pd.read_csv('estaciones-metro-2.csv')
metro = metro.drop(columns = ['Geometry', 'stop_id', 'stop_code', 'stop_desc', 'trip_heads', 'agency_id', 'Geopoint'])
metro.head()

,stop_name,stop_lat,stop_lon
0,Bellas Artes_1_2,19.435960,-99.141558
1,Canal del Norte_1,19.448667,-99.115895
2,Colegio Militar,19.448646,-99.171224
3,Buenavista_1_4,19.446158,-99.152899
4,Chapultepec,19.420256,-99.176588


Add the metro data points to the map:

In [6]:
for lat, lng, stop in zip(metro['stop_lat'], metro['stop_lon'], metro['stop_name']):
    label = '{}'.format(stop)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#63A3D3',
        fill=True,
        fill_color='#63A3D3',
        fill_opacity=0.7,
        parse_html=False).add_to(boroughs_map) 

boroughs_map

### Part 3: Foursquare data

Let's add the top 15 destinations in each borough to the map. First we save our credentials and create a function to retrieve the data from Foursquare:

In [10]:
CLIENT_ID = 'C3T3UYEQQS3TO0ZYE10HDFY1PHY121QLZKDWB2W2HUGSUYNG'
CLIENT_SECRET = 'CTFRAQY0RHNTWHXRRVHXE3PCMEZUP0B1DFTOSMMCN504N1FJ'
VERSION = '20190101'

Now we can find the top 15 venues for all boroughs using data from Foursquare:

In [11]:
LIMIT = 15

def getNearbyVenues(names):
    
    venues_list=[]
    for name in zip(names):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            name,
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Type']
    
    return(nearby_venues)

We'll store the data in a dataframe:

In [12]:
venues = getNearbyVenues(names=housing['Borough'])

('Tlalpan',)
('Tláhuac',)
('Venustiano Carranza',)
('Cuauhtémoc',)
('Iztapalapa',)
('Azcapotzalco',)
('Benito Juárez',)
('Cuajimalpa de Morelos',)
('Iztacalco',)
('Gustavo A. Madero',)
('Miguel Hidalgo',)
('Magdalena Contreras',)
('Coyoacán',)
('Milpa Alta',)
('Álvaro Obregón',)
('Xochimilco',)


View the dataframe:

In [13]:
venues.head()

,Venue,Venue Latitude,Venue Longitude,Venue Type
0,Bosque de Tlalpan,19.295363,-99.193256,Park
1,Superman - El Último Escape,19.295267,-99.209841,Theme Park Ride / Attraction
2,Medusa Steel Coaster,19.294227,-99.210888,Theme Park Ride / Attraction
3,La Castellana,19.286187,-99.157800,Liquor Store
4,Pista del Bosque De Tlalpan,19.295567,-99.193259,Trail


Finally, we can place the data on our map:

In [12]:
for lat, lng, venue, group in zip(venues['Venue Latitude'], venues['Venue Longitude'], venues['Venue'], venues['Venue Type']):
    label = '{}, {}'.format(venue, group)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#63D37C',
        fill=True,
        fill_color='#63D37C',
        fill_opacity=0.7,
        parse_html=False).add_to(boroughs_map)  

boroughs_map